# Getting started



---

Installing required libraries and Importing Packages

---



In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Creatng dataset from intents file

In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add tags to classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")


163 documents
23 classes ['', 'Clever', 'CourtesyGoodBye', 'CourtesyGreeting', 'CourtesyGreetingResponse', 'CurrentHumanQuery', 'Gali', 'Gossip', 'GreetingResponse', 'Jokes', 'Love', 'NameQuery', 'NotTalking2U', 'PodBayDoor', 'SelfAware', 'Shutup', 'Thanks', 'UnderstandQuery', 'age', 'goodbye', 'greeting', 'options', 'riddle']
144 unique lemmatized words ["'s", ',', 'a', 'about', 'adam', 'age', 'am', 'any', 'anyone', 'are', 'ask', 'asshole', 'aulad', 'aware', 'bay', 'bc', 'be', 'bella', 'bhenchod', 'bhosdike', 'bkl', 'bored', 'bosdike', 'bsdk', 'by', 'bye', 'call', 'can', 'cao', 'cheering', 'clever', 'communicating', 'comprendo', 'conscious', 'could', 'cya', 'date', 'day', 'do', 'doing', 'door', 'enough', 'for', 'friend', 'fuck', 'gaand', 'gandu', 'genious', 'get', 'girl', 'give', 'good', 'goodbye', 'gossip', 'got', 'great', 'greeting', 'have', 'hear', 'hello', 'help', 'helpful', 'hey', 'hi', 'hola', 'hope', 'how', 'hya', 'i', 'intelligent', 'is', 'it', 'joke', 'ki', 'know', 'later', '

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


# Building and tuning the model

Defining the model

In [ ]:
def make_model(mods):

  model=Sequential()
  model.add(Dense(mods.Int('units_1', min_value=16, max_value=1024,step=16),activation=mods.Choice("activation_1", ["relu","elu" ,"tanh","selu"]), input_shape=(len(train_x[0]),), kernel_regularizer=tf.keras.regularizers.l2(mods.Float('l2', 1e-6, 1e-2,sampling='log'))))
  model.add(Dropout(mods.Choice('rate_1', [ 0.0, 0.1, 0.2, 0.3, 0.4,0.5,0.6,0.7 ]))) 
  model.add(Dense(mods.Int('units_2', min_value=16, max_value=1024,step=16),activation=mods.Choice("activation_2", ["relu","elu" ,"tanh","selu"]),kernel_regularizer=tf.keras.regularizers.l2(mods.Float('l2', 1e-6, 1e-2,sampling='log'))))
  model.add(Dropout(mods.Choice('rate_2', [ 0.0, 0.1, 0.2, 0.3, 0.4,0.5,0.6,0.7 ]))) 
  model.add(Dense(mods.Int('units_3', min_value=16, max_value=1024,step=16),activation=mods.Choice("activation_3", ["relu","elu" ,"tanh","selu"]),kernel_regularizer=tf.keras.regularizers.l2(mods.Float('l2', 1e-6, 1e-2,sampling='log'))))
  model.add(Dropout(mods.Choice('rate_3', [ 0.0, 0.1, 0.2, 0.3, 0.4,0.5,0.6,0.7 ]))) 
  model.add(Dense(len(train_y[0]),activation=mods.Choice("activation_4", ["relu","elu" ,"tanh","selu"]),kernel_regularizer=tf.keras.regularizers.l2(mods.Float('l2', 1e-6, 1e-2,sampling='log'))))
  model.add(Dropout(mods.Choice('rate_4', [ 0.0, 0.1, 0.2, 0.3, 0.4,0.5,0.6,0.7 ]))) 
  sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
  model.summary()
  return model


Importing keras tuner and tuning the hyperparmeters

In [ ]:
import keras_tuner as kt
import datetime
# run parameter
log_dir = "logs/" + datetime.datetime.now().strftime("%m%d-%H%M")

tc = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    embeddings_freq=1,
    write_graph=True,
    update_freq='batch')
tuner=kt.RandomSearch(make_model,objective = 'accuracy', max_trials=100, overwrite=True )

tuner.search(np.array(train_x), np.array(train_y), epochs=200, batch_size=4, verbose=1, callbacks=[tc])
tuner.search_space_summary()
tuner.results_summary()

Trial 26 Complete [00h 02m 23s]
accuracy: 0.042944785207509995

Best accuracy So Far: 0.6809815764427185
Total elapsed time: 01h 11m 16s

Search: Running Trial #27

Value             |Best Value So Far |Hyperparameter
192               |176               |units_1
selu              |tanh              |activation_1
0.00019873        |9.8121e-05        |l2
0.7               |0.1               |rate_1
512               |912               |units_2
elu               |tanh              |activation_2
0.2               |0                 |rate_2
576               |560               |units_3
selu              |selu              |activation_3
0.6               |0                 |rate_3
selu              |selu              |activation_4
0.3               |0                 |rate_4

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 192)               27840     


In [ ]:
import tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir 'logs/'

In [ ]:
import matplotlib.pyplot as plt
def plt_metric(history, metric, title, has_valid=True):
    """Plots the given 'metric' from 'history'.

    Arguments:
        history: history attribute of History object returned from Model.fit.
        metric: Metric to plot, a string value present as key in 'history'.
        title: A string to be used as title of plot.
        has_valid: Boolean, true if valid data was passed to Model.fit else false.

    Returns:
        None.
    """
    plt.plot(history[metric])
    if has_valid:
        plt.plot(history["val_" + metric])
        plt.legend(["train", "validation"], loc="upper left")
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel("epoch")
    plt.show()

In [ ]:
best_trials = tuner.get_best_hyperparameters(num_trials=10)
i=0
for trial in best_trials:
    model = tuner.hypermodel.build(trial)
    history = model.fit(np.array(train_x), np.array(train_y), validation_split=0.2, epochs=500, batch_size=5, verbose=1 ,callbacks=[ec])
    # Plot the accuracy
    plt_metric(history=history.history, metric="accuracy", title="Model accuracy")
    # Plot the loss
    plt_metric(history=history.history, metric="loss", title="Loss")
    i+=1
    model.save(str(i))

In [ ]:
from keras.models import load_model
model = load_model('1')
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


In [ ]:
msg = input("you:")
while msg!="quit":
  res = chatbot_response(msg)
  print("ChatBot:"+res)
  
  msg = input("you:")